In [1]:
import pandas as pd
import numpy as np
import re
# NLP-specific packages: 
import nltk
from nltk.corpus import names
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from gensim import corpora
from nltk.corpus import names
from nltk.tokenize import word_tokenize
from nltk.text import Text  
from nltk.stem import PorterStemmer

import spacy.en
parser =  spacy.en.English()
nlp = spacy.load('en')

# output of multiple commands in a cell will be output at once.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# display up to 80 columns, this keeps everything visible
pd.set_option('display.max_columns', 80)
pd.set_option('expand_frame_repr', True)


In [3]:
# datapath = '~/intro_to_machine_learning/data'
datapath = '../../data'
full_df = pd.read_csv(datapath+'/df_module_3.csv', encoding='ISO-8859-1',low_memory=False)
# Only consider the first 1000 obs
df = full_df.iloc[:1000,:]

In [4]:
def remove_html_code(row):
    '''
    Purpose: Use a regex expression to remove any html code which is surruonded by angle brackets <>
    '''
    return re.sub("\<.*\>","",row)
def clean_sentences(text):
    '''
    Purpose:  Remove stopwords and anything that is not a letter.  Lemmatize the remaining words.
    '''
    text = text.lower()
    m = re.findall("[^a-z]", text)
    if m:
        for non_letter in m:
            text = text.replace(non_letter,"")
    
    tok = nlp(text)
    # If there
    if tok and text not in stopwords.words('english'):
        return tok[0].lemma_
    else:
        return ''

In [38]:
df['text'] = df['description.texts.en'].apply(lambda x: remove_html_code(x))
df['clean_text'] = df['text'].apply(lambda x: [clean_sentences(y) for y in x.split()])
df['clean_text'] = df['clean_text'].apply(lambda x: [y for y in x if y !=''])

/Users/j/virtualenvs/delta/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/j/virtualenvs/delta/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
# Create vectors out of the text descriptions.
sentences = []
for i, x in enumerate(df['clean_text']):
    sentences.append(gensim.models.doc2vec.LabeledSentence(words = x, tags = [str(i)]))

In [40]:
# Instantiate model.
doc2vec_model = gensim.models.Doc2Vec(sentences,workers=8)

In [53]:
def get_best_match_for_row(idx):
    '''
    Purpose: For the specified row, return the idx of the row with the most similair text description
    '''
    most_similair_idx = int(doc2vec_model.docvecs.most_similar(str(idx), topn=1)[0][0])
    most_similair_text = df.loc[most_similair_idx,'description.texts.en']
    return most_similair_idx, most_similair_text
    

To see how we did, we can print out the description for the first row, and print out the description for its best match.

In [65]:
idx = 0
most_similair_idx, most_similair_text = get_best_match_for_row(idx)
# view the original
df.loc[idx,'description.texts.en']

'Naomi is a married woman. She has five children. She operates a retail business where she sells vegetables and charcoal. She has been involved in this business for seven years. Her business is located in a good area, and her primary customers are locals. <br /><br />She will use the KES 30,000 loan to buy more stock of vegetables and charcoal. This is her first loan with SMEP Microfinance Bank.'

In [66]:
# view the most similar
most_similair_text

'Harrison is a married man and blessed with one child. He supports his family as well as other relatives, and he likes his business very much.<br /><br />He is a blacksmith by profession and is diligent in his work. He makes affordable cooking jikos for household use. He has been in this business for a long time and enjoys meeting the needs and serving people of his community.<br /><br />To grow his business, he is requesting a loan of KES 60,000 to buy iron sheets and clay. He will use his business profit to repay his loan in a short time. This is his third loan, and he pays well.<br /><br />His dream is to give his family a good future.'